In [73]:
import pandas as pd
from tqdm import tqdm
import pandas as pd
import kaleido

import matplotlib.pyplot as plt
%matplotlib inline

pd.options.plotting.backend = "plotly"

import plotly.express as px
import plotly.graph_objects as go

In [43]:
df = pd.read_csv('users_bio_perspective.csv', header=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639763 entries, 0 to 639762
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   'ids'               639763 non-null  object 
 1    'TOXICITY'         639763 non-null  float64
 2    'SEVERE_TOXICITY'  639763 non-null  float64
 3    'IDENTITY_ATTACK'  639763 non-null  float64
 4    'INSULT'           639763 non-null  float64
 5    'PROFANITY'        639763 non-null  float64
 6    'THREAT'           639763 non-null  float64
dtypes: float64(6), object(1)
memory usage: 34.2+ MB


C:\Users\shuvo\AppData\Local\Temp\ipykernel_7572\3437900853.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('users_bio_perspective.csv', header=0)


In [44]:
df.head()

,'ids','TOXICITY','SEVERE_TOXICITY','IDENTITY_ATTACK','INSULT','PROFANITY','THREAT'
0,2989319032,0.101285,0.060036,0.181704,0.067941,0.076990,0.160009
1,1042385216,0.375368,0.317461,0.609838,0.301542,0.311155,0.267791
2,144965060,0.695427,0.510649,0.800897,0.634811,0.508982,0.431851
3,18975463,0.310894,0.103242,0.595503,0.293295,0.156651,0.061779
4,635586076,0.562476,0.254738,0.803341,0.467187,0.313154,0.134482


In [45]:
cols = df.columns.to_list()
cols.pop(0)

for i in df.index:
    sum = df.loc[i, cols].sum()
    if sum > 1:
        print('Greater than 1')
        break

print('Done')

Greater than 1
Done


In [46]:
max = 0
col = ''
for c in cols:
    s = df.loc[:, c].max()
    if s > max:
        col = c
        max = s

print(f'Max: {max} for {col}')

Max: 0.998345 for  'INSULT'


In [47]:
df.max()

C:\Users\shuvo\AppData\Local\Temp\ipykernel_7572\1299571182.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.max()


 'TOXICITY'           0.997585
 'SEVERE_TOXICITY'    0.967473
 'IDENTITY_ATTACK'    0.987041
 'INSULT'             0.998345
 'PROFANITY'          0.997455
 'THREAT'             0.994093
dtype: float64

In [48]:
id_2_verdict_map = dict()
with open('users_bio_distilbert.csv', mode='r', encoding='utf8') as fin:
    fin.readline()
    for line in tqdm(fin):
        parts = line.strip().split(',')
        id_2_verdict_map[parts[0]] = parts[-1]

#id_2_verdict_map

1741912it [00:01, 970493.18it/s]


In [49]:
id = df.loc[0]['\'ids\'']
id
print(id_2_verdict_map.get(str(id)))

anger


In [51]:
all_neg_ids = set()
for [key, val] in id_2_verdict_map.items():
    if val in ['anger', 'fear', 'sadness']:
        all_neg_ids.add(key)

In [56]:
verdicts = []

for i in df.index:
    try:
        id = str(df.loc[i]['\'ids\''])
        if id in id_2_verdict_map:
            verdicts.append(id_2_verdict_map[id])
            all_neg_ids.discard(id)
        else:
            verdicts.append('other')

    except Exception:
        print(df.loc[i])


In [59]:
print(len(verdicts))
print(len(df))

df['verdict'] = verdicts

639763
639763


In [66]:
anger_data = df.loc[df.verdict == 'anger']
fear_data = df.loc[df.verdict == 'fear']
sad_data = df.loc[df.verdict == 'sadness']

In [75]:
anger_data

,'ids','TOXICITY','SEVERE_TOXICITY','IDENTITY_ATTACK','INSULT','PROFANITY','THREAT',verdict
0,2989319032,0.101285,0.060036,0.181704,0.067941,0.076990,0.160009,anger
1,1042385216,0.375368,0.317461,0.609838,0.301542,0.311155,0.267791,anger
2,144965060,0.695427,0.510649,0.800897,0.634811,0.508982,0.431851,anger
3,18975463,0.310894,0.103242,0.595503,0.293295,0.156651,0.061779,anger
4,635586076,0.562476,0.254738,0.803341,0.467187,0.313154,0.134482,anger
...,...,...,...,...,...,...,...,...
639758,1281589329039630336,0.834566,0.720307,0.428311,0.715006,0.843885,0.506163,anger
639759,993660792712163333,0.241703,0.128100,0.241558,0.214423,0.127449,0.174230,anger
639760,260033476,0.092878,0.025754,0.028447,0.049220,0.042975,0.083434,anger
639761,889910735073050627,0.142301,0.072376,0.094774,0.131713,0.079347,0.161846,anger


In [ ]:
figure = go.Figure()
figure.add_trace(go.Box(y=anger_data[' \'TOXICITY\''], name='TOXICITY'))
figure.add_trace(go.Box(y=anger_data[' \'SEVERE_TOXICITY\''], name='SEVERE_TOXICITY'))
figure.add_trace(go.Box(y=anger_data[' \'IDENTITY_ATTACK\''], name='IDENTITY_ATTACK'))
figure.add_trace(go.Box(y=anger_data[' \'INSULT\''], name='INSULT'))
figure.add_trace(go.Box(y=anger_data[' \'PROFANITY\''], name='PROFANITY'))
figure.add_trace(go.Box(y=anger_data[' \'THREAT\''], name='THREAT'))

figure.update_layout(
            title='Google Perspective API analysis of BIOs labeled as \'anger\' by DistilBERT',
            xaxis_title = 'Perspectives',
            yaxis_title = 'Probabilities',
            legend_title = 'Legends')

figure.show()

In [ ]:
figure = go.Figure()
figure.add_trace(go.Box(y=fear_data[' \'TOXICITY\''], name='TOXICITY'))
figure.add_trace(go.Box(y=fear_data[' \'SEVERE_TOXICITY\''], name='SEVERE_TOXICITY'))
figure.add_trace(go.Box(y=fear_data[' \'IDENTITY_ATTACK\''], name='IDENTITY_ATTACK'))
figure.add_trace(go.Box(y=fear_data[' \'INSULT\''], name='INSULT'))
figure.add_trace(go.Box(y=fear_data[' \'PROFANITY\''], name='PROFANITY'))
figure.add_trace(go.Box(y=fear_data[' \'THREAT\''], name='THREAT'))

figure.update_layout(
            title='Google Perspective API analysis of BIOs labeled as \'fear\' by DistilBERT',
            xaxis_title = 'Perspectives',
            yaxis_title = 'Probabilities',
            legend_title = 'Legends')

figure.show()

In [ ]:
figure = go.Figure()
figure.add_trace(go.Box(y=sad_data[' \'TOXICITY\''], name='TOXICITY'))
figure.add_trace(go.Box(y=sad_data[' \'SEVERE_TOXICITY\''], name='SEVERE_TOXICITY'))
figure.add_trace(go.Box(y=sad_data[' \'IDENTITY_ATTACK\''], name='IDENTITY_ATTACK'))
figure.add_trace(go.Box(y=sad_data[' \'INSULT\''], name='INSULT'))
figure.add_trace(go.Box(y=sad_data[' \'PROFANITY\''], name='PROFANITY'))
figure.add_trace(go.Box(y=sad_data[' \'THREAT\''], name='THREAT'))

figure.update_layout(
            title='Google Perspective API analysis of BIOs labeled as \'sadness\' by DistilBERT',
            xaxis_title = 'Perspectives',
            yaxis_title = 'Probabilities',
            legend_title = 'Legends')

figure.show()